In [ ]:
from tf_weldon import WeldonPooling

In [ ]:
base_model.summary()

In [1]:
import tensorflow as tf
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


model = ResNet50(include_top=False, weights='imagenet', pooling=None)
for i, layer in enumerate(model.layers):
    print(i, layer.name)
#model.summary()

/home/aamomeni/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 activatio

In [3]:
for i, layer in enumerate(model.layers[80:]):
    print(i, layer.name)

for layer in model.layers[:80]:
    layer.trainable = False
    
model.summary()

0 res4a_branch2a
1 bn4a_branch2a
2 activation_23
3 res4a_branch2b
4 bn4a_branch2b
5 activation_24
6 res4a_branch2c
7 res4a_branch1
8 bn4a_branch2c
9 bn4a_branch1
10 add_8
11 activation_25
12 res4b_branch2a
13 bn4b_branch2a
14 activation_26
15 res4b_branch2b
16 bn4b_branch2b
17 activation_27
18 res4b_branch2c
19 bn4b_branch2c
20 add_9
21 activation_28
22 res4c_branch2a
23 bn4c_branch2a
24 activation_29
25 res4c_branch2b
26 bn4c_branch2b
27 activation_30
28 res4c_branch2c
29 bn4c_branch2c
30 add_10
31 activation_31
32 res4d_branch2a
33 bn4d_branch2a
34 activation_32
35 res4d_branch2b
36 bn4d_branch2b
37 activation_33
38 res4d_branch2c
39 bn4d_branch2c
40 add_11
41 activation_34
42 res4e_branch2a
43 bn4e_branch2a
44 activation_35
45 res4e_branch2b
46 bn4e_branch2b
47 activation_36
48 res4e_branch2c
49 bn4e_branch2c
50 add_12
51 activation_37
52 res4f_branch2a
53 bn4f_branch2a
54 activation_38
55 res4f_branch2b
56 bn4f_branch2b
57 activation_39
58 res4f_branch2c
59 bn4f_branch2c
60 add_13


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Function


class WeldonPool2dFunction(Function):

    def __init__(self, kmax, kmin):
        super(WeldonPool2dFunction, self).__init__()
        self.kmax = kmax
        self.kmin = kmin

    def get_number_of_instances(self, k, n):
        if k <= 0:
            return 0
        elif k < 1:
            return round(k * n)
        elif k > n:
            return int(n)
        else:
            return int(k)

    def forward(self, input):
        # get batch information
        batch_size = input.size(0)
        num_channels = input.size(1)
        h = input.size(2)
        w = input.size(3)

        # get number of regions
        n = h * w

        # get the number of max and min instances
        kmax = self.get_number_of_instances(self.kmax, n)
        kmin = self.get_number_of_instances(self.kmin, n)

        # sort scores
        sorted, indices = input.new(), input.new().long()
        torch.sort(input.view(batch_size, num_channels, n), dim=2, descending=True, out=(sorted, indices))

        # compute scores for max instances
        self.indices_max = indices.narrow(2, 0, kmax)
        output = sorted.narrow(2, 0, kmax).sum(2).div_(kmax)

        if kmin > 0:
            # compute scores for min instances
            self.indices_min = indices.narrow(2, n-kmin, kmin)
            output.add_(sorted.narrow(2, n-kmin, kmin).sum(2).div_(kmin)).div_(2)

        # save input for backward
        self.save_for_backward(input)
        # return output with right size
        return output.view(batch_size, num_channels)

    def backward(self, grad_output):

        # get the input
        input, = self.saved_tensors

        # get batch information
        batch_size = input.size(0)
        num_channels = input.size(1)
        h = input.size(2)
        w = input.size(3)

        # get number of regions
        n = h * w

        # get the number of max and min instances
        kmax = self.get_number_of_instances(self.kmax, n)
        kmin = self.get_number_of_instances(self.kmin, n)

        # compute gradient for max instances
        grad_output_max = grad_output.view(batch_size, num_channels, 1).expand(batch_size, num_channels, kmax)
        grad_input = grad_output.new().resize_(batch_size, num_channels, n).fill_(0).scatter_(2, self.indices_max, grad_output_max).div_(kmax)

        if kmin > 0:
            # compute gradient for min instances
            grad_output_min = grad_output.view(batch_size, num_channels, 1).expand(batch_size, num_channels, kmin)
            grad_input_min = grad_output.new().resize_(batch_size, num_channels, n).fill_(0).scatter_(2, self.indices_min, grad_output_min).div_(kmin)
            grad_input.add_(grad_input_min).div_(2)

        return grad_input.view(batch_size, num_channels, h, w)


class WeldonPool2d(nn.Module):

    def __init__(self, kmax=1, kmin=None):
        super(WeldonPool2d, self).__init__()
        self.kmax = kmax
        self.kmin = kmin
        if self.kmin is None:
            self.kmin = self.kmax

    def forward(self, input):
        return WeldonPool2dFunction(self.kmax, self.kmin)(input)

    def __repr__(self):
        return self.__class__.__name__ + ' (kmax=' + str(self.kmax) + ', kmin=' + str(self.kmin) + ')'


In [ ]:
from_idx= 0
    
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[from_idx:]:
    print(layer)
    layer.trainable = True
    

In [ ]:
model.summary()

In [ ]:
from config import Config
config = Config()
from TCGA_Datasets import TCGA_Dataset

dataset = TCGA_Dataset(config)

samples = dataset._partition[0]['test']
labels = dataset._partition[1]['test']
X, y = dataset.convert_to_arrays(samples, labels, phase = 'test', size = 1)

In [ ]:
len(y) / config.sampling_size_val

In [ ]:
def patch_to_image(y_patches, proba=True):
        
        if proba == True:
            y_image = np.array([np.mean(y_patches[i*config.sampling_size_val:(i+1)*config.sampling_size_val])for i in range(int(len(y_patches)/config.sampling_size_val))]).reshape((-1,1))
        else:
            y_image = np.array([np.mean(y_patches[i*config.sampling_size_val:(i+1)*config.sampling_size_val])>0.5 for i in range(len(y_patches/config.sampling_size_val))]).reshape((-1,1))
        y_image = y_image.flatten()
        return y_image

In [ ]:
import numpy as np 
len(patch_to_image(y, proba = False))
len(y)
#len(y/config.sampling_size_val)

In [ ]:
import pandas as pd 
import numpy as np 
import os 
from sklearn.preprocessing import LabelEncoder

images_ids = os.listdir('/labs/gevaertlab/data/cedoz/patches_448')
labels_ids = list(pd.read_excel('TCGA-MICCAI-Patients.xlsx',  index_col = 'Patient').index)
labels = np.intersect1d(labels_ids, images_ids)
table = pd.read_excel('TCGA-MICCAI-Patients.xlsx', index_col = 'Patient')

c = table[table.index.isin(labels)]
le = LabelEncoder()

binarized_data = c.apply(le.fit_transform)
print(le.classes_)
test_data = pd.read_table('MICCAI_labels.txt', index_col = 0, delim_whitespace = True, header = 0)
y_test = test_data.apply(le.fit).values.flatten()
#binarized_data.values.flatten()
test_data.apply(le.fit_transform).values.flatten()

In [ ]:
import pandas as pd 
df = pd.read_excel('TCGA-MICCAI-Patients.xlsx',index_col = 'Patient')
df = df[df.index.isin(labels)]
le = LabelEncoder()
binarized_data = df.apply(le.fit_transform).values.flatten()
test_data = pd.read_table('MICCAI_labels.txt', index_col = 0, delim_whitespace = True, header = 0)
ids_test = test_data.index
le.classes_

In [ ]:
output_data = pd.read_table('MICCAI_labels.txt', index_col = 0, delim_whitespace = True, header = 0)
output_data.apply(le.fit_transform).values.flatten()


In [ ]:
import numpy as np
a = [1,2,3,4,5,6,7,8,9,10]
np.repeat(a, 8)



In [ ]:
from config import Config

In [ ]:
config = Config()

In [ ]:
from tensorflow.contrib.slim.python.slim.nets import resnet_v1
import tensorflow.contrib.slim as slim


In [ ]:

def get_binarized_data():

    output_data = pd.read_table('MICCAI_labels.txt', index_col = 0, delim_whitespace = True, header = 0)
    binarized_data = output_data.apply(lambda x: LabelBinarizer().fit_transform(x)[:, 0], axis=0)
    return binarized_data

def get_partition(self):

    samples = os.listdir("/labs/gevaertlab/data/MICCAI/patches_448")
    np.random.shuffle(samples)
    idx_val = int((1-self.config.val_size)*len(samples))
    idx_test = int((1 - config.test_size) * len(samples))
    train_samples, val_samples, test_samples = np.split(samples, [idx_val, idx_test])
    train_samples, val_samples, test_samples = list(train_samples), list(val_samples), list(test_samples)
    train_ids = self.get_ids(train_samples)
    self.partition = {'train': train_samples, 'val': val_samples, 'test': test_samples}

def get_ids(self, samples):

    ids = []
    for sample in samples:
        patches = os.listdir("/labs/gevaertlab/data/MICCAI/patches_448/%s" % sample)
        patches = np.random.choice(patches, size=self.config.sampling_size_train, replace=True)
        for patch in patches:
            ID = "%s/patches_%d/%s/%s"%(self.config.data_path, self.config.patch_size, sample, patch)
            ids.append(ID)

    return ids

def get_labels(self):

    samples = os.listdir("/labs/gevaertlab/data/MICCAI/patches_448"%(self.config.data_path,self.config.patch_size))
    self.labels = {}
    data = self.get_binarized_data()
    for i in data.columns:
        self.labels[i] = {}
        for s in samples:
            self.labels[i][s] = data.loc[i,s]

In [ ]:
import pandas as pd 
from sklearn.preprocessing import LabelBinarizer
import os
import numpy as np
from config import Config
from PIL import Image


config = Config()


def get_binarized_data():

    output_data = pd.read_table('MICCAI_labels.txt', index_col = 0, delim_whitespace = True, header = 0)
    binarized_data = output_data.apply(lambda x: LabelBinarizer().fit_transform(x)[:, 0], axis=0)
    return binarized_data

def get_labels():
    samples = os.listdir("/labs/gevaertlab/data/MICCAI/patches_448")
    labels = {}
    data = get_binarized_data()
    for i in data.columns:
        labels[i] = {}
        for s in samples:
            labels[i][s] = data.loc[s,i]
    return labels

def get_ids(samples):

    ids = []
    for sample in samples:
        patches = os.listdir("/labs/gevaertlab/data/MICCAI/patches_448/%s" % sample)
        patches = np.random.choice(patches, size= config.sampling_size_train, replace=True)
        for patch in patches:
            ID = "/labs/gevaertlab/data/MICCAI/patches_448/%s/%s"%(sample, patch)
            ids.append(ID)

    return ids


def get_partition():

    samples = os.listdir("/labs/gevaertlab/data/MICCAI/patches_448")
    np.random.shuffle(samples)
    idx_val = int((1-config.val_size)*len(samples))
    idx_test = int((1 - config.test_size) * len(samples))
    train_samples, val_samples, test_samples = np.split(samples, [idx_val, idx_test])
    train_samples, val_samples, test_samples = list(train_samples), list(val_samples), list(test_samples)
    train_ids = get_ids(train_samples)
    partition = {'train': train_samples, 'val': val_samples, 'test': test_samples}
    return partition
    
def convert_to_arrays(samples, labels):
        
        X, ids = [], []
        for sample in samples:
            patches = os.listdir("/labs/gevaertlab/data/MICCAI/patches_448/%s"%sample)
            patches = np.random.choice(patches, size=config.sampling_size_val, replace=True)
            for patch in patches:
                ID = "/labs/gevaertlab/data/MICCAI/patches_448/%s/%s"% (sample, patch)
                ids.append(ID)
                img = Image.open(ID)
                image = np.array(img)[:,:,:3]
                X.append(image)  
        X = np.asarray(X)
        
        for label in labels.keys():
            y_label = []
            for ID in ids:
                sample = ID.split('/')[-2]
                y_label.append(labels[label][sample])
            y = np.asarray(y_label)

        return X, y

In [ ]:
samples =  get_partition()['train']
labels = get_labels()
convert_to_arrays(samples,labels)